In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","8g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .getOrCreate()

[ WARN] Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
[ WARN] Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import time

# get the start time
st = time.time()

surl = 'neo4j://localhost:7692'
spassword = 'password'
turl='neo4j+s://5b2c1911.databases.neo4j.io'
tpassword = 'yQF9RToGeogn06ONl4k9mEppEAHHmipmtX0Qk9x-UVQ'

In [3]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Career)<-[r]-(j:JobPosting) return c.name as name, j.id as jobid") \
  .load()

In [4]:
career = df.select('name').groupBy(df.name).count()

In [5]:
import pyspark.sql.functions as f
career = career.select(f.col('name'), f.col('count').alias('numpost'))

In [6]:
career = career.withColumn("timestamp", f.current_timestamp()).alias("timestamp")

In [7]:
career.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "name")\
    .option("labels",":Career")\
    .save()

In [8]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Career)<-[r1]-(j:JobPosting)-[r2]->(pl:ProgrammingLanguage) return c.name as careername, pl.name as name ") \
  .load()

In [9]:
pl = df.dropDuplicates(["name"]).select(df.name)
pl.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "name")\
    .option("labels",":ProgrammingLanguage")\
    .save()

In [10]:
career_pl = df.dropDuplicates(['careername','name'])

In [11]:
career_pl = df.groupBy('careername','name').count()

In [12]:
career_pl = career_pl.select(f.col('careername'), f.col('name'), f.col('count').alias('numPl'))

In [13]:
career_pl.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'NEED_PL')\
    .option("relationship.properties",'numPl') \
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Career')\
    .option('relationship.source.node.keys','careername:name')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':ProgrammingLanguage')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [14]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Career)<-[r1]-(j:JobPosting)-[r2]->(t:Tool) return c.name as careername, t.name as name") \
  .load()

In [15]:
tool = df.dropDuplicates(["name"]).select(df.name)
tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "name")\
    .option("labels",":Tool")\
    .save()

In [16]:
career_tool = df.dropDuplicates(['careername','name'])
career_tool = df.groupBy('careername','name').count()
career_tool = career_tool.select(f.col('careername'), f.col('name'), f.col('count').alias('numTool'))

In [17]:
career_tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'NEED_TOOL')\
    .option("relationship.properties",'numTool') \
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Career')\
    .option('relationship.source.node.keys','careername:name')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Tool')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [18]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Career)<-[r1]-(j:JobPosting)-[r2]->(t:Framework) return c.name as careername, t.name as name") \
  .load()

In [19]:
fram = df.dropDuplicates(["name"]).select(df.name)
fram.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "name")\
    .option("labels",":Framework")\
    .save()

In [20]:
career_fram = df.dropDuplicates(['careername','name'])
career_fram = df.groupBy('careername','name').count()
career_fram = career_fram.select(f.col('careername'), f.col('name'), f.col('count').alias('numFw'))

career_fram.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'NEED_FW')\
    .option("relationship.properties",'numFw') \
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Career')\
    .option('relationship.source.node.keys','careername:name')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Framework')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [21]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Career)<-[r1]-(j:JobPosting)-[r2]->(t:Platform) return c.name as careername, t.name as name") \
  .load()

In [22]:
plat = df.dropDuplicates(["name"]).select(df.name)
plat.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "name")\
    .option("labels",":Platform")\
    .save()

In [23]:
career_plat= df.dropDuplicates(['careername','name'])
career_plat = df.groupBy('careername','name').count()
career_plat = career_plat.select(f.col('careername'), f.col('name'), f.col('count').alias('numPf'))

career_plat.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'NEED_PF')\
    .option("relationship.properties",'numPf') \
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Career')\
    .option('relationship.source.node.keys','careername:name')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Platform')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [24]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Career)<-[r1]-(j:JobPosting)-[r2]->(t:Knowledge) return c.name as careername, t.name as name") \
  .load()

In [25]:
know = df.dropDuplicates(["name"]).select(df.name)
know.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "name")\
    .option("labels",":Knowledge")\
    .save()

In [26]:
career_know= df.dropDuplicates(['careername','name'])
career_know = df.groupBy('careername','name').count()
career_know = career_know.select(f.col('careername'), f.col('name'), f.col('count').alias('numKl'))

career_know.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'NEED_KL')\
    .option("relationship.properties",'numKl') \
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Career')\
    .option('relationship.source.node.keys','careername:name')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Knowledge')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [27]:
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 47.776694536209106 seconds
